In [9]:
%config IPCompleter.greedy=True
import numpy as np
import matplotlib.pyplot as plt
import math

# Proyecto correlacion
## Equipo 40 
(Luis Antonio Dominguez Alejo, Luis Enrique Navarro Morales,Hugo Cesar Mondragon Basurto & Eric Javier Trejo Rodriguez)
- Factor de compresibilidad isotermico
$$ c_{g} = \frac{1}{p}-\frac{1}{z}(\frac{\delta{z}}{\delta{p}})$$ 
$$ c_g*p_{pc} = c_{pr} = \frac{1}{p_{pr}}-	[ \,\frac{1}{z^2T_{pr}}(\frac{\frac{\delta{z}}{\delta{\rho_{pr}}}}{1+((\frac{\rho_{pr}}{z})(\frac{\delta{z}}{\delta{\rho_{pr}}}))})	] \, $$
- Factor de desviacion a partir de `DPR`o `DAK`
$$"DPR" :z=1+(A_1+\frac{A_2}{T_{sr}}+\frac{A_3}{T_{sr}^3})\rho_r+(A_4+\frac{A_5}{T_{sr}})\rho_r^2+(\frac{A_5A_6\rho_r^5}{T_{sr}})+A_7(1+A_8\rho_r^2)(\frac{\rho_r^2}{T_{sr}^3})(exp(-A_8\rho_r^2)) $$
$$ "DAK" :z = 1+(A_1+\frac{A_2}{T_{sr}}+\frac{A_3}{T_{sr}^3}+\frac{A_4}{T_{sr}^4}+\frac{A_5}{T_{sr}^5})\rho_r+(A_6+\frac{A_7}{T_{sr}}+\frac{A_8}{T_{sr}^2})\rho_r^2-A_9(\frac{A_7}{T_{sr}}+\frac{A_8}{T_{sr}^2})\rho_r^5 + A_{10}(1+A_{11}\rho_r^2)(\frac{\rho_r^2}{T_{sr}^3})(exp(-A_{11}\rho_r^2) $$ 
- Factor de volumen de gas
$$ B_g = \frac{\frac{nzRT}{p}}{\frac{nz_eRT_e}{P_e}} $$

## DPR

In [2]:
def z_DPR(p_pr,t_pr,tol = 1e-13):
    
    A1 = 0.31506237
    A2 = -1.0467099
    A3 = -0.57832720
    A4 =  0.53530771
    A5 = -0.61232032
    A6 = -0.10488813
    A7 = 0.68157001
    A8 = 0.68446549
    
    T1 = A1 + A2 / t_pr + A3 / t_pr**3
    T2 = A4 + A5 / t_pr
    T3 = A5 * A6 / t_pr
    T4 = A7 / t_pr**3
    T5 = 0.27 * p_pr / t_pr
    
     
    def  f(rho_r):
        f = 1 + T1*rho_r + T2 * rho_r**2 + T3 * rho_r**5 + (T4 * rho_r**2 * (1 + A8 * rho_r**2)) * np.exp(-A8 * rho_r**2) - T5 / rho_r
        return f
    
    def f_prima(rho_r):
        f_prima = T1 + 2 * T2 * rho_r + 5 * T3 * rho_r**4 + 2 * T4 * rho_r * np.exp(-A8 * rho_r**2) * ((1 + 2 * A8 * rho_r**2) - A8 * rho_r**2 * (1 + A8 * rho_r**2)) + T5 / rho_r**2
        return f_prima 
    
    rho_r0 =  0.27 * p_pr / t_pr 
    rho_rk = rho_r0
    i = -1
    while True:
        if (abs(f(rho_rk)) < tol):
            break 
        
        rho_rk1 = rho_rk - f(rho_rk) / f_prima(rho_rk)
        delta = abs(rho_rk-rho_rk1)
        
        if (delta < tol):
            break 
        else:
            rho_rk = rho_rk1
            i = i+1
    z = 0.27 * p_pr / (rho_rk * t_pr)
    print(rho_rk)
    return z, rho_rk

In [3]:
p_pr = float(input("¿Cual es la presion pseudoreducida?\n"))
t_pr = float(input("¿Cual es la temperatura pseudoreducida?\n"))

¿Cual es la presion pseudoreducida?
5.3
¿Cual es la temperatura pseudoreducida?
1.34


In [4]:
z1 = z_DPR(p_pr,t_pr)
rho_r = z1[1]
z = z1[0]

1.4005119858525445


In [5]:
z

0.7625143222970109

In [6]:
print("El factor de desviacon:\n z=",z1[0])
print("\nLa densidad reducida es:\n rho_r=",z1[1])

El factor de desviacon:
 z= 0.7625143222970109

La densidad reducida es:
 rho_r= 1.4005119858525445


## DAK

In [10]:
def z_DAK(p_pr,t_pr):
    
    e = 1e-15
    i = 0 
      
    rho_r = (0.27*p_pr)/t_pr

    while True:
        A1 = 0.3265 ; A2 = -1.0700 ; A3 = -0.5339 ;  A4 = 0.01569 ; A5 = -0.05165
        A6 = 0.5475 ; A7 = -0.7361 ; A8 = 0.1844 ; A9 = 0.1056 ; A10 = 0.6134  ; A11 = 0.7210
        R1 = A1 + A2/t_pr + A3/t_pr**3 + A4/t_pr**4 + A5/t_pr**5
        R2 = (0.27*p_pr)/t_pr
        R3 = A6 + A7/t_pr + A8/t_pr**2 
        R4 = A9*(A7/t_pr+A8/t_pr**2)
        R5 = A10 / t_pr**3
        f = R1*rho_r - R2/rho_r + R3*rho_r**2 - R4*rho_r**5 + A10*(1+A11*rho_r**2)*((rho_r**2)/(t_pr**3))*math.exp(-A11*rho_r)
        f_prima = R1+R2/rho_r**2 + 2*R3*rho_r - 5*(R4)*rho_r**4 + 2*(R5)*rho_r*math.exp(-A11*rho_r)*((1+2*A11*rho_r**3)-A11*rho_r**2*(1+A11*rho_r**2))
        rho_n = rho_r -f/f_prima
        error = abs(rho_n-rho_r)
        if error < e:
            break
            
        rho_r = rho_n
        print("iteracion=",i,"\nerror=",error,"\nrho:",rho_r)
        i = i+1
        z = (0.27*p_pr)/(rho_r*t_pr)
        
    return (z,rho_r)

In [11]:
df=z_DAK(p_pr,t_pr)
z11 = df[0]
rho_r1 = df[1]

iteracion= 0 
error= 1.2308248845466767 
rho: 2.2987353323078707
iteracion= 1 
error= 0.41254824724267425 
rho: 1.8861870850651965
iteracion= 2 
error= 0.12757625594693223 
rho: 1.7586108291182643
iteracion= 3 
error= 0.00678394129671811 
rho: 1.7653947704149824
iteracion= 4 
error= 0.0011685282162530175 
rho: 1.7642262421987294
iteracion= 5 
error= 0.00019422760330711775 
rho: 1.7644204698020365
iteracion= 6 
error= 3.24861665004228e-05 
rho: 1.764387983635536
iteracion= 7 
error= 5.427952042946771e-06 
rho: 1.764393411587579
iteracion= 8 
error= 9.070867852400966e-07 
rho: 1.7643925045007938
iteracion= 9 
error= 1.5158251120972466e-07 
rho: 1.764392656083305
iteracion= 10 
error= 2.5330948894008998e-08 
rho: 1.764392630752356
iteracion= 11 
error= 4.233050798774229e-09 
rho: 1.7643926349854069
iteracion= 12 
error= 7.073845953442515e-10 
rho: 1.7643926342780223
iteracion= 13 
error= 1.1821099654696354e-10 
rho: 1.7643926343962333
iteracion= 14 
error= 1.9754420321760335e-11 
rho: 1.7

In [12]:
print("El factor de desviacon:\n z=",z11)
print("\nLa densidad reducida es:\n rho_r=",rho_r1)

El factor de desviacon:
 z= 0.6052566911428261

La densidad reducida es:
 rho_r= 1.7643926343793077


## Factor de compresibilidad isotermico
`Mattar, Brar & Aziz`

In [13]:
def cg(p_pr,t_pr,z,rho_r,p_pc):
    
    A1 = 0.31506237
    A2 = -1.0467099
    A3 = -0.57832720
    A4 =  0.53530771
    A5 = -0.61232032
    A6 = -0.10488813
    A7 = 0.68157001
    A8 = 0.68446549
    
    dz_drho = A1 + A2/t_pr + A3/t_pr**3 + 2*(A4 + A5/t_pr)*rho_r + 5*A5*A6*(rho_r**4/t_pr) + (((2*A7*rho_r)/(t_pr**3))*(1+A8*rho_r**2-(A8*rho_r**2)**2)*np.exp(-A8*rho_r**2))
    
    c_r = 1/p_pr - (0.27/(z**2*t_pr))*((dz_drho)/(1+(rho_r/z)*(dz_drho)))
    
    c_g = c_r/p_pc
    
    return c_g    

In [14]:
p_pc = float(input("¿Cual es la presion pseudocritica?\n" ))

¿Cual es la presion pseudocritica?
680


In [15]:
#cg(5.60,1.35,0.78970,1.41826,680)
c_g = cg(p_pr,t_pr,z,rho_r,p_pc)

In [16]:
c_g1 = c_g*10**6
print("La compresibilidad isotermica del gas:\n C_g=",c_g1,"x10^(-6) [psi^-1]")

La compresibilidad isotermica del gas:
 C_g= 138.41754913156208 x10^(-6) [psi^-1]


## Factor volumetrico de gas

In [17]:
def B_g(z,T,p):
    Bg = 0.02827*((z*T)/(p))
    return Bg

In [18]:
p = float(input("¿Cual es la presion?\n"))
T = float(input("¿Cual es la temperatura?\n"))

¿Cual es la presion?
3810
¿Cual es la temperatura?
654


In [19]:
Bg=B_g(z,T,p)

In [20]:
print("EL factor volumetrico del gas:\n Bg=",Bg,"[PCE/PCY]")

EL factor volumetrico del gas:
 Bg= 0.0037002118238672094 [PCE/PCY]
